<font size = 10> **Regridding velocity variables** </font>  
The ACCESS-OM2-01 model has velocity variables available as outputs (`u` for zonal velocity and `v` for meridional velocity). However, unlike any other outputs available in the ACCESS-OM2-01 model, these velocity variables do not share the same grid. This means that we need to regrid these variables to match all other model outputs before we can extract data to use as input for our species distribution model.

# Setting working directory

In [2]:
import os
os.chdir(os.path.expanduser('~/Chapter2_Crabeaters/Scripts'))

# Loading relevant libraries

In [3]:
import UsefulFunctions as uf
import xarray as xr
# import pandas as pd
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
# import cartopy.crs as ccrs
from dask.distributed import Client
import cosima_cookbook as cc
import xesmf as xe

# Creating dictionary of variables
The values in this dictionary will be used several times throughout this notebook.

In [4]:
varDict = {'model': 'ACCESS-OM2-01',
           #ACCESS-OM2-01 cycle 4 (1958-2018)
           'exp': '01deg_jra55v140_iaf_cycle4',
           #ACCESS-OM2-01 cycle 4 extension (2018-2022)
           'exp_ext': '01deg_jra55v140_iaf_cycle4_jra55v150_extension',
           #Temporal resolution
           'freq': '1 monthly',
           #Name of variable in ACCESS-OM2-01 model
           'var_mod': 'v',
           #Short name
           'short_var': 'vel_lat',
           #Output folder
           'base_folder_out': '/g/data/v45/la6889/Chapter2_Crabeaters/Velocity_Variables/',
           #File name base
           'base_file_out': 'monthly_lat_'}

# Paralellise work

In [3]:
client = Client()

# Load velocity variable to be regridded

In [10]:
#Create a COSIMA cookbook session
session = cc.database.create_session()

#Loading data from fourth cycle (1978 to 2020)
var_df = uf.getACCESSdata_SO(varDict['var_mod'], '1978-01', '2020-01', 
                              freq = varDict['freq'], ses = session, minlat = -80,
                              exp = varDict['exp'], ice_data = False)
#Loading data from fourth cycle extension (2019 to 2022)
var_df_ext = uf.getACCESSdata_SO(varDict['var_mod'], '2019-01', '2023-01', 
                              freq = varDict['freq'], ses = session, minlat = -80,
                              exp = varDict['exp_ext'], ice_data = False)
#Concatenating both data arrays into one
var_df = xr.concat([var_df, var_df_ext], dim = 'time')

#Transforming longitudes so their range is +/-180 degrees
var_df = uf.corrlong(var_df)
#Removing duplicate variable
del var_df_ext

## Extracting surface data

In [7]:
#Select surface layer for depth and ensuring only data between 1978 and 2022 is selected
var_df_s = var_df.isel(st_ocean = 0).sel(time = slice('1978-01', '2022-12'))
#Remove depth (st_ocean) from coordinates as it is no longer useful
var_df_s = var_df_s.squeeze().drop('st_ocean')
#Rechunk data to facilitate data manipulation
var_df_s = var_df_s.chunk({'time': 12, 'yu_ocean': 513, 'xu_ocean': 200}) 
#Checking results
var_df_s

<xarray.DataArray 'v' (time: 540, yu_ocean: 714, xu_ocean: 3600)>
dask.array<rechunk-merge, shape=(540, 714, 3600), dtype=float32, chunksize=(12, 513, 200), chunktype=numpy.ndarray>
Coordinates:
  * xu_ocean  (xu_ocean) float64 -180.0 -179.9 -179.8 ... 179.7 179.8 179.9
  * yu_ocean  (yu_ocean) float64 -79.99 -79.95 -79.9 ... -45.14 -45.07 -45.0
  * time      (time) datetime64[ns] 1978-01-16T12:00:00 ... 2022-12-16T12:00:00
Attributes: (12/13)
    long_name:      j-current
    units:          m/sec
    valid_range:    [-10.  10.]
    cell_methods:   time: mean
    time_avg_info:  average_T1,average_T2,average_DT
    coordinates:    geolon_c geolat_c
    ...             ...
    ncfiles:        ['/g/data/cj50/access-om2/raw-output/access-om2-01/01deg_...
    contact:        Andrew Kiss
    email:          andrew.kiss@anu.edu.au
    created:        2022-04-27
    description:    0.1 degree ACCESS-OM2 global model configuration under in...
    notes:          Run configuration and history: https://github.com/COSIMA/...

### Saving surface data
We will save files for each year as an intermediary output in case files are needed in the future.

In [40]:
#Ensure base folder exists, if not, create it
os.makedirs(varDict['base_folder_out'], exist_ok = True)

#Looping through each year and saving outputs as netcdf files
for yr, da in var_df.groupby('time.year'):
    file_out = os.path.join(varDict['base_folder_out'], varDict['base_file_out']+f'surf_{yr}.nc')
    da.to_netcdf(file_out)

'/g/data/v45/la6889/Chapter2_Crabeaters/Velocity_Variables/monthly_surf_lat_20.nc'

## Extracting bottom data
Bottom data needs more processing that surface data. In this case, we need to identify the deepest bin with data for each grid cell.

In [11]:
#Create a mask identifying grids cell with values as 1 or where there are NAs as 0
mask_2d = xr.where(~np.isnan(var_df.isel(time = 0)), 1, np.nan)
#Calculate cumulative sum across depth in mask
mask_2d = mask_2d.cumsum('st_ocean').where(~np.isnan(var_df.isel(time = 0)))
#Identify the depth for the maximum values of cumulative sums
mask_2d = xr.where(mask_2d == mask_2d.max('st_ocean'), 1, np.nan)
#Apply mask to data
var_2d = (mask_2d*var_df).sum('st_ocean')
#Reorder coordinates and ensure only data between 1978 and 2022 remains
var_df = var_2d.transpose('time', 'yu_ocean', 'xu_ocean').sel(time = slice('1978-01', '2022-12'))
#Rechunk data to facilitate data manipulation
var_df = var_df.chunk({'time': 12, 'yu_ocean': 513, 'xu_ocean': 200}) 
#Checking results
var_df

<xarray.DataArray 'v' (time: 540, yu_ocean: 714, xu_ocean: 3600)>
dask.array<rechunk-merge, shape=(540, 714, 3600), dtype=float64, chunksize=(12, 513, 200), chunktype=numpy.ndarray>
Coordinates:
  * xu_ocean  (xu_ocean) float64 -180.0 -179.9 -179.8 ... 179.7 179.8 179.9
  * yu_ocean  (yu_ocean) float64 -79.99 -79.95 -79.9 ... -45.14 -45.07 -45.0
  * time      (time) datetime64[ns] 1978-01-16T12:00:00 ... 2022-12-16T12:00:00

### Saving bottom data
We will save files for each year as an intermediary output in case files are needed in the future.

In [ ]:
for yr, da in var_df.groupby('time.year'):
    file_out = os.path.join(varDict['base_folder_out'], varDict['base_file_out']+f'bottom_{yr}.nc')
    da.to_netcdf(file_out)

/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.01/lib/python3.9/site-packages/dask/array/reductions.py:649: RuntimeWarning: All-NaN slice encountered
  return np.nanmax(x_chunk, axis=axis, keepdims=keepdims)
/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.01/lib/python3.9/site-packages/dask/array/reductions.py:649: RuntimeWarning: All-NaN slice encountered
  return np.nanmax(x_chunk, axis=axis, keepdims=keepdims)
/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.01/lib/python3.9/site-packages/dask/array/reductions.py:649: RuntimeWarning: All-NaN slice encountered
  return np.nanmax(x_chunk, axis=axis, keepdims=keepdims)
2023-08-12 13:44:17,806 - distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
2023-08-12 13:44:18,825 - distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
2023-08-12 13:44:25,276 - distributed.utils_perf - WARNING - full garbage collections took 10% CPU

# Regridding velocity data

## Loading sample target grid
We will load a sample of the grid used by all other outputs in the ACCESS-OM2-01 model. We will use this as our target grid. We will load the `area_t` variable, which contains the area of each grid cell.

In [4]:
#Load grid for the Southern Ocean
grid = cc.querying.getvar(varDict['exp'], 'area_t', session, n = 1).sel(yt_ocean = slice(-80, -45))
#Transforming longitudes so their range is +/-180 degrees
grid = uf.corrlong(grid)
#We will now create and attach a mask of land area
grid['mask'] = xr.where(~np.isnan(grid), 1, 0)
#We will rename the coordinates so the regridder can recognise it
grid = grid.rename({'xt_ocean': 'lon', 'yt_ocean': 'lat'})
#Checking results
grid.mask.plot()

<xarray.DataArray 'area_t' (yt_ocean: 713, xt_ocean: 3600)>
dask.array<getitem, shape=(713, 3600), dtype=float32, chunksize=(513, 720), chunktype=numpy.ndarray>
Coordinates:
  * xt_ocean  (xt_ocean) float64 -179.9 -179.8 -179.7 ... 179.8 179.9 180.0
  * yt_ocean  (yt_ocean) float64 -79.97 -79.93 -79.88 ... -45.18 -45.11 -45.04
Attributes:
    long_name:     tracer cell area
    units:         m^2
    valid_range:   [0.e+00 1.e+15]
    cell_methods:  time: point
    coordinates:   geolon_t geolat_t
    ncfiles:       ['/g/data/cj50/access-om2/raw-output/access-om2-01/01deg_j...
    contact:       Andrew Kiss
    email:         andrew.kiss@anu.edu.au
    created:       2022-04-27
    description:   0.1 degree ACCESS-OM2 global model configuration under int...
    notes:         Run configuration and history: https://github.com/COSIMA/0...

## Loading velocity data from memory 

In [1]:
#Getting list of files to be regridded
files_to_regrid = sorted(glob(os.path.join(varDict['base_folder_out'], 'regridded',varDict['short_var'], '*.nc')))

## Loading a single file from memory 
We just need a single timestep to calculate the regridder that will be applied to all velocity files.

In [ ]:
#Loading a single time step
var_df = xr.open_dataarray(files_to_regrid[0]).isel(time = 0)
#Adding a mask for land areas
var_df['mask'] = xr.where(~np.isnan(var_df), 1, 0).squeeze().drop('time')
#We will rename the coordinates so the regridder can recognise it
var_df = var_df.rename({'xu_ocean': 'lon', 'yu_ocean': 'lat'})
#Checking result
var_df

## Calculate regridder
We will apply the bilinear method which performs well and has a relatively short processing time.

In [7]:
#We provide the file to be regridded followed by our target grid
regridder = xe.Regridder(var_df, grid, method = 'bilinear')
#Saving regridder
regridder.to_netcdf(os.path.join(varDict['base_folder_out'], 'regridder.nc'))

xESMF Regridder 
Regridding algorithm:       bilinear 
Weight filename:            bilinear_714x3600_713x3600.nc 
Reuse pre-computed weights? False 
Input grid shape:           (714, 3600) 
Output grid shape:          (713, 3600) 
Periodic in longitude?      False

### **Optional step**: Loading regridder for use later
We need to load the regridder file and use it as `weights` when we calculate the regridder once. This takes less than the original regridder calculation.  
Remember that to calculate the regridder, we only need a single time step for the data we want to regrid.

In [11]:
#Load regridder
regrid = xr.open_dataset(os.path.join(varDict['base_folder_out'], 'regridder.nc'))
#Once again, we provide a single time step of the original data to be regriddded
regridder = xe.Regridder(var_df, grid, method = 'bilinear', weights = regrid)

## Regridding data
Once we have the regridder ready, we can apply it to the velocity dataset. We will do this for the yearly files containing surface and bottom velocity information.

In [50]:
#Define folder for regridded data
folder_out = os.path.join(varDict['base_folder_out'], 'regridded', varDict['short_var'])
#Ensure it exists
os.makedirs(folder_out, exist_ok = True)

#Loop through all the files in the list
for f in files_to_regrid:
    #Load as data array
    var_df = xr.open_dataarray(f)
    #Create land mask before regridding
    var_df['mask'] = xr.where(~np.isnan(var_df.isel(time = 0)), 1, 0).squeeze().drop('time')
    #Renaming coordinates so the regridder can recognise them
    var_df = var_df.rename({'xu_ocean': 'lon', 'yu_ocean': 'lat'})
    #Applying regridder
    reg_var_df = regridder(var_df)
    #Renaming coordinates so they match coordinates for all other ACCESS-OM2-01 outputs
    reg_var_df = reg_var_df.rename({'lon': 'xt_ocean', 'lat': 'yt_ocean'})
    #Get year contained in data array
    yr = np.unique(reg_var_df.time.dt.year)[0]
    #Renaming variable in data array and saving output
    if 'bottom' in f:
        reg_var_df.name = varDict['short_var']+'_bottom_msec'
        file_out = os.path.join(folder_out, 'monthly_bottom_' + varDict['short_var'] + f'_{yr}.nc')
    elif 'surf' in f:
        reg_var_df.name = varDict['short_var']+'_surf_msec'
        file_out = os.path.join(folder_out, 'monthly_surf_' + varDict['short_var'] + f'_{yr}.nc')
    #Saving output as netcdf
    reg_var_df.to_netcdf(file_out)

'/g/data/v45/la6889/Chapter2_Crabeaters/Velocity_Variables/regridded/vel_lat'